In [1]:
import numpy as np
import operator

In [2]:
#分类函数
#testData 测试向量
#dataSet 训练数据集
#label    训练标签
#k       knn的k值选定
def classify0(testData, dataSet, label, k):
    dataSize = dataSet.shape[0]
    diffMat = np.tile(testData, (dataSize, 1)) - dataSet
    sqDiff = diffMat**2
    sqDiffSum = sqDiff.sum(axis=1)
    distance = sqDiffSum**0.5
    sortIndex = distance.argsort()
    classCnt = {}
    for i in range(k):
        vote = label[sortIndex[i]]
        classCnt[vote] = classCnt.get(vote, 0) + 1
    sortClassCnt = sorted(classCnt.items(), key=operator.itemgetter(1), reverse=True)
#     print(sortClassCnt)
    return sortClassCnt[0][0]

In [3]:
#自定义训练数据
dataSet = np.array([
    [1.0, 1.1],
    [1.0, 1.1],
    [0, 0],
    [0, 0.1]
])

label = ['a', 'a', 'b', 'b']

In [4]:
#测试分类函数
classify0([0.1,0.1], dataSet, label, 3)

'b'

In [5]:
#约会网站数据
#将文件中的数据转为矩阵
def file2matrix(filename):
    fr = open(filename)
    arrayOfLine = fr.readlines()
    numOfLine = len(arrayOfLine)
    returnMat = np.zeros((numOfLine, 3))
    classLabelVector = []
    index = 0
    for line in arrayOfLine:
        line = line.strip()
        lineForm = line.split('\t')
        returnMat[index, :] = lineForm[0:3]
        classLabelVector.append(int(line[-1]))
        index += 1
    return returnMat, classLabelVector

In [6]:
#normaliz
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals, (m, 1))
    normDataSet = normDataSet / np.tile(ranges, (m, 1))
    return normDataSet, ranges, minVals

In [7]:
#test
def datingClassTest():
    hoRatio = 0.1
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]
    numTestVecs = int(m*hoRatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:], normMat[numTestVecs:m,:], datingLabels[numTestVecs:m], 3)
        print("the classifier came back with: %d, the real answer is: %d"%(classifierResult, datingLabels[i]))


In [8]:
datingClassTest()

the classifier came back with: 3, the real answer is: 3
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answe

In [9]:
#给定数据来预测
def classifyPerson():
    resultList = ['not at all', 'in small doses', 'in large doses']
    percentTats = float(input("percentage of time spent playing video games ?"))
    ffMiles = float(input("frequent flier miles earned per year ?"))
    iceCream = float(input("liters of inc cream consumed per year?"))
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    inArr = np.array([ffMiles, percentTats, iceCream])
    classifierResult = classify0((inArr - minVals)/ranges, normMat, datingLabels, 3)
    print(classifierResult)
    print("you will probably like this person :", resultList[classifierResult-1])

In [10]:
classifyPerson()

percentage of time spent playing video games ?9
frequent flier miles earned per year ?40000
liters of inc cream consumed per year?1
3
you will probably like this person : in large doses


In [11]:
classifyPerson()

percentage of time spent playing video games ?10
frequent flier miles earned per year ?10000
liters of inc cream consumed per year?0.6
2
you will probably like this person : in small doses


In [12]:
classifyPerson()

percentage of time spent playing video games ?1.4
frequent flier miles earned per year ?26000
liters of inc cream consumed per year?0.8
1
you will probably like this person : not at all


In [13]:
#handwriting number
def img2vector(filename):
    returnVect = np.zeros((1, 1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])
    return returnVect

In [14]:
import os
import sys
def handwritingClassTest():
    hwLabels = []
    trainingList = os.listdir('trainingDigits')
    m = len(trainingList)
    trainingMat = np.zeros((m, 1024))
    for i in range(m):
        fileNameStr = trainingList[i]
        fileStr = fileNameStr.split('.')[0] #获取文件名
        classNumStr = int(fileStr.split('_')[0])   #获取类别
        hwLabels.append(classNumStr)
        trainingMat[i, :] = img2vector('trainingDigits/%s'%fileNameStr)  #获取特征值
    
    testList = os.listdir('testDigits')
    errorCnt = 0.0
    mTest = len(testList)
    for i in range(mTest):
        fileNameStr = testList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        testData = img2vector('testDigits/%s'%fileNameStr)
        res = classify0(testData, trainingMat, hwLabels, 3)
        print("the classifier came back with : %d, the real answer is: %d"%(res, classNumStr))
        if(res != classNumStr):
            errorCnt += 1.0
    print("the total number of errors is : %d"%errorCnt)
    print("the accuracy is : %f"%(1 - errorCnt/float(mTest)))

In [15]:
handwritingClassTest()

the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with : 0, the real answer is: 0
the classifier came back with :

the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with : 1, the real answer is: 1
the classifier came back with :

the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with : 3, the real answer is: 3
the classifier came back with :

the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with : 4, the real answer is: 4
the classifier came back with :

the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with : 6, the real answer is: 6
the classifier came back with :

the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with : 7, the real answer is: 7
the classifier came back with :

the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with : 9, the real answer is: 9
the classifier came back with :

In [ ]:
def handwritingClassTest1():
    hwLabels = []
    trainingList = os.listdir('trainingDigits')
    m = len(trainingList)
    trainingMat = np.zeros((m, 1024))
    for i in range(m):
        fileNameStr = trainingList[i]
        fileStr = fileNameStr.split('.')[0] #获取文件名
        classNumStr = int(fileStr.split('_')[0])   #获取类别
        hwLabels.append(classNumStr)
        trainingMat[i, :] = img2vector('trainingDigits/%s'%fileNameStr)  #获取特征值
    
    testList = os.listdir('testDigits')
    errorCnt = 0.0
    mTest = len(testList)
    for i in range(mTest):
        fileNameStr = testList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        classIndex = int(fileStr.split('_')[1])
        testData = img2vector('testDigits/%s'%fileNameStr)
        res = classify0(testData, trainingMat, hwLabels, 3)
#         print("the classifier came back with : %d, the real answer is: %d"%(res, classNumStr))
        if(res != classNumStr):
            errorCnt += 1.0
            print("the wrong predict index is: %d"%classIndex)
            print("the classifier came back with : %d, the real answer is: %d"%(res, classNumStr))
    print("the total number of errors is : %d"%errorCnt)
    print("the accuracy is : %f"%(1 - errorCnt/float(mTest)))

In [ ]:
handwritingClassTest1()

the wrong predict index is: 86
the classifier came back with : 7, the real answer is: 1
the wrong predict index is: 11
the classifier came back with : 9, the real answer is: 3


In [ ]:
# ![jupyter](./jupyter_logo.png)

预测错误的数字

预测为7，实际为1

![jupyter](./1-86.jpg)

预测为9，实际为3
说实话，我真的看不出来是3

![jupyter](./3-11.jpg)